In [1]:
import time
import sm
import PanelOLSData
import pandas as pd
from PanelOLSData import append_data
from PanelOLSData import add_countrycol
from linearmodels import PanelOLS

In [2]:
# No extra rows above the the dataframe column row, use skiprows!
rawdata_protests  = pd.read_csv('./data/mmALL_073120_csv.csv')
rawdata_homicides = pd.read_excel('./data/data_cts_intentional_homicide.xlsx', skiprows = 2)
rawdata_cpi       = pd.read_excel('./data/CPI.xlsx', skiprows = 1)
rawdata_gdp       = pd.read_excel('./data/API_NY_GDP.PCAP_CD_DS2_en_excel_v2_4770383.xls', skiprows = 3)
rawdata_ineq      = pd.read_csv('./data/DP_LIVE_15012023210214655.csv')
countrycodes_df   = pd.read_csv('./data/country_codes.csv', sep = ';')[
    ['Country', 'Alpha-2 code', 'Alpha-3 code', 'Numeric']
]

In [3]:
start = time.time()

# Contries and years not available in the starting point (in this case the target dataset) are 
# ignored in other datasets. To maximize data accross all datasets the starting point should be 
# the biggest dataset.
dataset_protests = append_data(rawdata_protests, 'y', 'country', 'year', 'protest')
dataset_protests = append_data(rawdata_gdp, 'x', 'Country Name', '', 'GDP', dataset_protests)

# Homicides dataset has multiple indicators, dimensions, sexes and age groups, so it must be filtered!
rawdata_homicides = rawdata_homicides[rawdata_homicides['Unit of measurement'] == 'Rate per 100,000 population']
rawdata_homicides = rawdata_homicides[rawdata_homicides['Indicator'] == 'Victims of intentional homicide']
rawdata_homicides = rawdata_homicides[rawdata_homicides['Dimension'] == 'Total']
rawdata_homicides = rawdata_homicides[rawdata_homicides['Sex'] == 'Total']
rawdata_homicides = rawdata_homicides[rawdata_homicides['Age'] == 'Total']
rawdata_homicides = rawdata_homicides.rename(columns={'VALUE': 'Homicides'})
dataset_protests  = append_data(rawdata_homicides, 'y', 'Country', 'Year', 'Homicides', dataset_protests)

# The CPI dataset is a mix of x and y df types. We create a new Y CPI dataset before calling the funcion
rawdata_cpi_y = pd.DataFrame()
rawdata_cpi   = rawdata_cpi.rename(columns = {'CPI Score 2013': 'CPI score 2013', 
                                              'CPI Score 2012': 'CPI score 2012'})
i = 0
for country in rawdata_cpi.Country.unique():
    for year in [country.split(' ')[2] for country in rawdata_cpi.columns if 'score' in country.lower()]:
        rawdata_cpi_y.loc[i, 'country'] = country
        rawdata_cpi_y.loc[i, 'year'] = int(year)
        rawdata_cpi_filt = rawdata_cpi[rawdata_cpi.Country == country]['CPI score '+ str(year)]
        rawdata_cpi_y.loc[i, 'CPI'] = rawdata_cpi_filt[rawdata_cpi_filt.index[0]]
        i += 1
dataset_protests = append_data(rawdata_cpi_y, 'y', 'country', 'year', 'CPI', dataset_protests)

# For the Gini data we need to get country names from Alpha-3 codes
inner_merged_total = add_countrycol(rawdata_ineq, 'LOCATION', 'Alpha-3 code', countrycodes_df)
inner_merged_total = inner_merged_total.rename(columns={'Value': 'GINI'})
dataset_protests   = append_data(inner_merged_total, 'y', 'Country', 'TIME', 'GINI', dataset_protests)

end = time.time()
print(str(end - start) + ' seconds.')

dataset_protests


Starting new dataset...
----------------------------------------------------------------------------------------------------------------------------------------------------------------------
There were missed samples in due to names miss matching or missing years in the following countries:
{'Austria', 'Macedonia', 'Bosnia', 'Germany West', 'Czechoslovakia', 'USSR', 'Moldova', 'Croatia', 'Belarus', 'Russia', 'Czech Republic', 'South Sudan', 'Ukraine', 'Serbia and Montenegro', 'Latvia', 'Kosovo', 'Slovenia', 'Germany East', 'Azerbaijan', 'Montenegro', 'Timor Leste', 'Estonia', 'Slovak Republic', 'Yugoslavia', 'Serbia'}
----------------------------------------------------------------------------------------------------------------------------------------------------------------------
There were missed samples in due to names miss matching or missing years in the following countries:
{'Macedonia', 'South Korea', 'Syria', 'Congo Kinshasa', 'Iran', 'Venezuela', 'Turkey', 'Bosnia', 'Germany 

,country,year,protest,GDP,Homicides,CPI,GINI
0,Canada,1990.0,1.0,21448.361960,2.385506,NaN,2.1
1,Canada,1991.0,1.0,21768.343294,2.703593,NaN,2.1
2,Canada,1992.0,1.0,20879.848330,2.597043,NaN,2.2
3,Canada,1993.0,1.0,20121.161253,2.192801,NaN,2.1
4,Canada,1994.0,1.0,19935.381458,2.065090,NaN,2.1
...,...,...,...,...,...,...,...
5141,Papua New Guinea,2016.0,1.0,2332.697480,NaN,28.0,NaN
5142,Papua New Guinea,2017.0,1.0,2495.131383,NaN,29.0,NaN
5143,Papua New Guinea,2018.0,0.0,2584.298769,NaN,28.0,NaN
5144,Papua New Guinea,2019.0,0.0,2593.804650,NaN,28.0,NaN


In [10]:
# PanelOLS call of Protest in GDP and Homicides
#w1 = dataset_protests.drop('CPI', axis = 1).drop('GINI', axis = 1).set_index(['country', 'year'])
#mod = PanelOLS(w1['protest'], w1[['GDP','Homicides']], entity_effects=True)

# PanelOLS call of Protest in GDP, Homicides and CPI
#w1 = dataset_protests.drop('GINI', axis = 1).set_index(['country', 'year'])
#mod = PanelOLS(w1['protest'], w1[['GDP','Homicides', 'CPI']], entity_effects=True)
# PanelOLS call of Protest in GDP, Homicides, CPI and GINI
w1 = dataset_protests.dropna().set_index(['country', 'year'])
mod = PanelOLS(w1['protest'], w1[['GDP','Homicides', 'CPI', 'GINI']], entity_effects=True)

result = mod.fit(cov_type='clustered', cluster_entity=True)
display(result.summary)

Dep. Variable:,protest,R-squared:,0.0051
Estimator:,PanelOLS,R-squared (Between):,0.5462
No. Observations:,191,R-squared (Within):,0.0051
Date:,"Tue, Jan 17 2023",R-squared (Overall):,0.4148
Time:,13:54:13,Log-likelihood,-63.899
Cov. Estimator:,Clustered,,
,,F-statistic:,0.2022
Entities:,29,P-value,0.9368
Avg Obs:,6.5862,Distribution:,"F(4,158)"
Min Obs:,1.0000,,
Max Obs:,9.0000,F-statistic (robust):,0.4062
